# Deal with writing a csv file for the language modeling for radiology reports

Author: Geeticka Chauhan

In [2]:
%load_ext autoreload

Requirements: 
Make sure to `pip install spacy` 
and `pip install scispacy` with the relevant model being en_core_sci_md from `https://allenai.github.io/scispacy/`

In [3]:
%autoreload

import os
import sys
sys.path.append('../..')
import pandas as pd
import joint_img_txt.data.text.lm_utils as lm_utils
import re
import joint_img_txt.data.text.preprocess as preprocess

import glob
from tqdm import tqdm
import re
from pyexpat import ExpatError

in_dir = '/data/vision/polina/projects/chestxray/data_v2/reports/' 
# Replace the above with the location of MIMIC-CXR .txt reports
out_dir = '/data/vision/polina/projects/chestxray/geeticka/pre-processed/' 
# Replace the above with the output directory for where you would like the csv file of all reports located
# this is where the outputted csv files will be located inside the directory reports_list/

def in_res(filename): return os.path.join(in_dir, filename)
def out_res(filename): return os.path.join(out_dir, filename)
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
if not os.path.exists(out_res('lm_reports')):
    os.mkdir(out_res('lm_reports'))

In [692]:
reports_df = lm_utils.get_reports_df(in_dir)

Total files to read: 224672  from dir:  /data/vision/polina/projects/chestxray/data_v2/reports/


File s51431102.txt involves appending to the dictionary value list
File s58090273.txt involves appending to the dictionary value list
File s51966946.txt has importance sections empty and following section with too many space
File s51966946.txt involves appending to the dictionary value list
File s52762865.txt involves appending to the dictionary value list
File s59385570.txt has importance sections empty and following section with too many space
File s59385570.txt involves appending to the dictionary value list
File s59420146.txt has importance sections empty and following section with too many space
File s59420146.txt involves appending to the dictionary value list
File s50106655.txt has importance sections empty and following section with too many space
File s50106655.txt involves appending to the dictionary value list
File s57295412.txt involves appending to the dictionary value list
File s57350021.txt has importance sections empty and following section with too many space
File s573

File s53747822.txt has importance sections empty and following section with too many space
File s53747822.txt involves appending to the dictionary value list
File s57319606.txt involves appending to the dictionary value list
File s50058765.txt involves appending to the dictionary value list
File s50833007.txt involves appending to the dictionary value list
File s50269150.txt involves appending to the dictionary value list
File s53072658.txt has importance sections empty and following section with too many space
File s53072658.txt involves appending to the dictionary value list
File s57387939.txt involves appending to the dictionary value list
File s53649363.txt has importance sections empty and following section with too many space
File s53649363.txt involves appending to the dictionary value list
File s53245159.txt involves appending to the dictionary value list
File s55346214.txt involves appending to the dictionary value list
File s50835258.txt involves appending to the dictionary v

File s58067260.txt involves appending to the dictionary value list
File s57271119.txt involves appending to the dictionary value list
File s54346445.txt involves appending to the dictionary value list
File s54346445.txt involves appending to the dictionary value list
File s56274071.txt involves appending to the dictionary value list
File s57595015.txt involves appending to the dictionary value list
File s58501303.txt involves appending to the dictionary value list
File s54407986.txt has importance sections empty and following section with too many space
File s54407986.txt involves appending to the dictionary value list
File s54977408.txt involves appending to the dictionary value list
File s55285472.txt has importance sections empty and following section with too many space
File s55285472.txt involves appending to the dictionary value list
File s58141654.txt has importance sections empty and following section with too many space
File s58141654.txt involves appending to the dictionary v

File s51850376.txt involves appending to the dictionary value list
File s53590982.txt involves appending to the dictionary value list
File s59281202.txt has importance sections empty and following section with too many space
File s59281202.txt involves appending to the dictionary value list
File s53718151.txt involves appending to the dictionary value list
File s59698776.txt involves appending to the dictionary value list
File s54374353.txt has importance sections empty and following section with too many space
File s54374353.txt involves appending to the dictionary value list
File s55486770.txt involves appending to the dictionary value list


In [693]:
lm_utils.write_dataframe(reports_df, out_res('lm_reports/original_reports.csv'))

In [694]:
reports_df_copy = lm_utils.read_dataframe(out_res('lm_reports/original_reports.csv'))

Double check if the reports read from csv is the same as that written to disk previously. For sanity checking. 

In [695]:
lm_utils.check_equality_of_written_and_read_df(reports_df, reports_df_copy)

(True, True)

# The following sections are optional, and may help you explore MIMIC-CXR reports further. This helped me make the code to process the reports more accurate.

## Let's look at how many keys (i.e. report section headings) are possible

In [696]:
keys = []
for row, val in original_reports_df.iterrows():
    if type(val['report']['final_report']) is not str:
        keys.extend(list(val['report']['final_report'].keys()))

In [697]:
set(keys)

{']history',
 ']impression',
 'abdomen',
 'abdomenchest',
 'abnormal',
 'above_the_hemidiaphragm_with_mass_effect',
 'absence_of_any_prior_chest_imaging',
 'accession_number',
 'acute_cardriopulm_diseaase_surg',
 'acute_pathology_surg',
 'addendum',
 'addendum_the_correct_report_should_read_as_follows',
 'additional_clinical_history_provided',
 'additional_findings',
 'again_demonstrated_are',
 'air',
 'all_worsened_since_deid_are',
 'allowing_for_these_limitations',
 'ambulation_issues_surg',
 'and_read',
 'anesthesia',
 'answered',
 'any_acute_intrapulmonary_process_surg',
 'ap',
 'ap_and_lateral_chest_radiograph',
 'ap_and_lateral_chest_radiographs',
 'ap_and_lateral_radiographs_of_the_chest',
 'ap_and_lateral_views_of_the_chest',
 'ap_chest',
 'ap_chest_at_deid_and_compared_to_deid_for',
 'ap_chest_radiograph',
 'ap_chest_reviewed_in_the_absence_of_prior_chest_imaging',
 'ap_chest_reviewed_in_the_absence_of_prior_chest_radiographs',
 'ap_film',
 'ap_frontal_and_lateral_chest_radiog

## Investigating empty files

According to my algorithm, only s54168089.txt is completely empty i.e. has no text. The ones that are in below list just don't have impressions and findings sections

In [21]:
total_files_to_read = glob.glob(in_dir + '*.txt')

In [22]:
len(reports_df)

224671

In [23]:
len(total_files_to_read)

224672

In [24]:
chexpert_files = glob.glob('/data/vision/polina/projects/chestxray/work_space_v2/report_processing/chexpert_labels/' + '*.csv')

In [25]:
def get_file_names(filelist):
    filenames = [os.path.splitext(os.path.basename(a))[0] for a in filelist]
    return filenames

In [26]:
total_filenames = get_file_names(total_files_to_read)
chexpert_filenames = get_file_names(chexpert_files)

In [27]:
empty_files = set(total_filenames) - set(chexpert_filenames)

In [28]:
with open('empty_files.txt', 'w') as empty_f:
    for filename in empty_files:
        empty_f.write(filename +'.txt'+ '\n')
# empty_files

The above files do not contain findings or impressions sections, and basically contain reason_for_examination sections. 

In [30]:
print("The following files contain findings or impressions section. Warning: some of the finding or impression might actually be empty but because the word was present, the algorithm thought it was there")
for file in empty_files:
    report_dict = lm_utils.get_report_dictionary(in_res(file + '.txt'))
    if report_dict and ('findings' in report_dict['final_report'].keys() or 'impression' in report_dict['final_report'].keys()):
        print("File %s.txt"%file)

The following files contain findings or impressions section. Warning: some of the finding or impression might actually be empty but because the word was present, the algorithm thought it was there
File s53822563.txt
File s55884447.txt
File s56869432.txt
File s52197607.txt
File s58393138.txt
File s59094925.txt
File s59363654.txt
File s53801626.txt
File s58851430.txt
File s59954972.txt
File s51704764.txt
The file /data/vision/polina/projects/chestxray/data_v2/reports/s54168089.txt is empty
File s53356173.txt
File s55460408.txt
File s58137144.txt
File s53627605.txt
File s51966946.txt
File s55664890.txt
File s50276465.txt
File s57484509.txt
File s53240872.txt
File s50487727.txt
File s58049533.txt
File s59285278.txt
File s51067117.txt
File s55354071.txt
File s53935525.txt
File s59385570.txt
File s54374353.txt
File s50336284.txt
File s50804446.txt
File s59569833.txt
File s58364589.txt
File s59532776.txt
File s53283173.txt
File s58794078.txt
File s59185554.txt
File s51394967.txt
File s5905297

## Various different tests

In [31]:
reports_df.iloc[0]['report']

{'final_report': {'chest': 'Frontal and lateral views',
  'comparison': 'None.',
  'examination': '',
  'findings': 'The lungs are clear without focal consolidation. The lungs are relatively hyperinflated, suggesting underlying chronic obstructive pulmonary disease. No pleural effusion or pneumothorax is seen. The cardiac and mediastinal silhouettes are unremarkable.',
  'impression': 'No acute cardiopulmonary process. If high clinical concern for pulmonary lesion, chest CT is more sensitive in detecting subtle lesions.',
  'indication': 'History: M with failure to thrive and weight loss, history of smoking // Evaluate for mass',
  'technique': 'Chest: Frontal and Lateral'}}

In [32]:
total_files_to_read = glob.glob(in_dir + '*.txt')
print('Total files to read:' , len(total_files_to_read) , ' from dir: ' , in_dir)

Total files to read: 224672  from dir:  /data/vision/polina/projects/chestxray/data_v2/reports/


In [33]:
file = in_res('s50000028.txt')
with open(file, 'r') as text_file:
    for line in text_file.readlines():
        line = line.strip()
        line = line.replace('_', '')
        print(line)

WET READ:   7:51 PM
6.2 x 5 cm mass projecting to the superior segment of the left lower lobe,
concerning for malignancy particularly given calcified pleural plaques.
Contrast enhanced CT recommended for further evaluation.  d/ 

WET READ VERSION #1

FINAL REPORT
CLINICAL HISTORY:  Cough, evaluate for pneumonia.

CHEST, PA AND LATERAL

COMPARISON FILM:  .

A mass is present in the superior segment of the left lower lobe and therefore
malignancy must be considered.  Elsewhere, the left lung appears clear.  There
is no effusion. Calcified pleural plaque is present in the right mid zone.
The right lung appears clear.

Some tracheal displacement to the right is present at the thoracic inlet
probably due to thyroid, but lymph nodes should also be considered.

IMPRESSION:  Left lung mass.  CT should be performed.


In [34]:
a = 'FINDINGS: sdjasjdja: jdsajdajs: adasdsas'
# m = re.search('(?<=.*:).*', a)
m = re.match('.*:.*', a) # check if the sentence contains quotes, most likely a section
section_name = a.split(':')[0]
m = re.search('(?<=%s:).*'%section_name, a)
# a.split(':')[0]
full_line = m.group(0).strip()

In [35]:
full_line

'sdjasjdja: jdsajdajs: adasdsas'

In [37]:
lm_utils.get_report_dictionary(in_res('s50817290.txt'))

{'final_report': {'no_section': 'REASON FOR EXAMINATION: Hypoxemia. AP radiograph of the chest was reviewed in comparison to . The ET tube tip is approximately 3 cm above the carina. The NG tube tip is in the stomach. Heart size and mediastinum are unremarkable. Lungs are well aerated with unchanged scarring in the right apex, chronic as well as still present minimal bibasilar atelectasis better appreciated on the CT torso. Overall, no substantial interval change to explain hypoxemia has been demonstrated on the current radiograph.'}}

In [632]:
original_reports_df = reports_df.copy()

In [682]:
# filename = ''
# filename = original_reports_df.iloc[0]['filename']
filename = 's59159499.txt'

In [683]:
report = ''                                                                                            
origin_sec = []                                                                                        
report_dict = original_reports_df[original_reports_df['filename'] == filename]['report'].values[0]     
sections_to_find = ['finding', 'impression', 'conclusion', 'recommendation']                           
values_found = [] # a list of all values found within the dictionary; possibly a list of lists         
for section in sections_to_find:                                                                       
    keys_found = []                                                                                    
    sections_found = []                                                                                
    final_report = report_dict['final_report']                                                         
    for s in final_report.keys():                                                                      
        if section in s:                                                                               
            keys_found.append(s)                                                                       
            sections_found.append(section)                                                                    
    if len(keys_found) > 0:                                                                            
        values_needed = [final_report[key] for key in keys_found]                                      
        # only add the values for the non duplicated keys                                              
        for val in values_needed:                                                                      
            if val not in values_found:                                                                
                values_found.append(val)                                                               
        for sec in sections_found:                                                                     
            if sec not in origin_sec:                                                                  
                origin_sec.append(sec)                                                                 
for val in values_found:                                                                               
    if type(val) is list:                                                                              
        report += ' '.join(val) + ' '                                                                  
    else: 
        report += val + ' '                                                                            
 #if any('finding' in s for s in report_dict['final_report'].keys()) and report_dict['final_report']['findings']:
 #    origin_sec.append('findings')                                                                     
 #    report += report_dict['final_report']['findings'] + ' '.                                          
 #if 'impression' in report_dict['final_report'].keys() and report_dict['final_report']['impression']:  
 #    origin_sec.append('impression')                                                                   
 #    report += ' ' + report_dict['final_report']['impression']                                         
report = lm_utils.remove_whitespace(report)                                                                     
# reports.append(report)                                                                                 
# origin_sections.append(origin_sec)   

In [684]:
report

''

In [685]:
report_dict['final_report']

{'ap_chest_at_deid_and_compared_to_deid_for': 'Tip of the right PICC line is at a level 6.5 cm inferior to the carina, and would need to be withdrawn 3 cm to place it in the low SVC. Lungs clear and heart size normal. No pleural abnormality. Esophagus is mildly distended with air to the level of the carina. Cardiomediastinal and hilar silhouettes are otherwise unremarkable',
 'comparison': '',
 'findings': '',
 'history': 'DEID -year-old man with infection.',
 'impression': ''}

In [686]:
import re 
line = 'AP chest at DEID and compared to DEID for:'
re.match('^[a-zA-z]+(([\s]{1,2}|/)[a-zA-Z]+)*:.*', line) # is basically saying let all pass that have spaces between and ;

<_sre.SRE_Match object; span=(0, 42), match='AP chest at DEID and compared to DEID for:'>